# Python XFoil Interface

As the article was written, I compiled Mark Drela's latest version of *XFoil* using **gfortran** on my Mac, and ran my airfoil analyses using the command line. In the middle of the project, I came across a version of *XFoil* put together by researchers at the *DAR Corporation*. This version removed the graphics components from the original code, and converted the code to a more modern version of *Fortran*. The end result is a library of routines that can be called from *Python*. 

In this section, we will walk through building an example library of *Fortran* code and accessing it from *Python*.

In [3]:
%%writefile ../tmp/double.f90

function double(n) result(result)
  implicit none
  integer, intent(in) :: n
  integer :: result

  result = n * 2
end function double

Overwriting ../tmp/double.f90


In [4]:
!gfortran -c ../tmp/double.f90 -o ../tmp/double.o

In [6]:
!gfortran ../tmp/double.o -dynamiclib -o ../tmp/double.dylib

Now, let's try to load this library.

First, we find the **tmp** directory:

In [8]:
import os
libdir = os.path.abspath('../tmp')
libpath = os.path.join(libdir,'double.dylib')

In [9]:
lib_ext = libpath[libpath.rfind('.'):]
lib_ext

'.dylib'

We need to access a few routines from the *Python* **ctypes** library, then we can load the library:

In [ ]:
from ctypes import cdll, c_int, POINTER, byref
lib = cdll.LoadLibrary(libpath)

*Fortran passes arguments as references. We need to set the argument and result types:

In [23]:
lib.double_.argtypes = [POINTER(c_int)]
lib.double_.restype = c_int

Now we can access the fortran function. We cannot just pass a Python data value, we need to convert the *Python* value into a **ctype**, and use the **byref** function to prepare it for *Fortran*:

In [24]:
n = c_int(2)
print(lib.double_(byref(n)))

4


Now that this process works, let's try to process an array of values and get a value back:


In [25]:
%%writefile ../tmp/vector_norm.f90

function vector_norm(n,vec) result(norm)
  implicit none
  integer, intent(in) :: n
  real, intent(in) :: vec(n)
  real :: norm

  norm = sqrt(sum(vec**2))

end function vector_norm

Writing ../tmp/vector_norm.f90


In [27]:
!python -m numpy.f2py -c ../tmp/vector_norm.f90 -m norm1

running build
running config_cc
INFO: unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
INFO: unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
INFO: build_src
INFO: building extension "norm1" sources
INFO: f2py options: []
INFO: f2py:> /var/folders/ws/5h_x4h713cg8w0jmrm5_tfzc0000gn/T/tmpv7hb703u/src.macosx-12-x86_64-3.10/norm1module.c
creating /var/folders/ws/5h_x4h713cg8w0jmrm5_tfzc0000gn/T/tmpv7hb703u/src.macosx-12-x86_64-3.10
Reading fortran codes...
	Reading file '../tmp/vector_norm.f90' (format:free)
Post-processing...
	Block: norm1
			Block: vector_norm
Post-processing (stage 2)...
Building modules...
    Building module "norm1"...
		Creating wrapper for Fortran function "vector_norm"("vector_norm")...
        Constructing wrapper function "vector_norm"...
          norm = vector_norm(vec,[n])
    Wrote C/API module "norm1" to file "/var/folders/ws/5h_x4h713cg8w0jmrm5_tfzc000

INFO: gfortran:f77: /var/folders/ws/5h_x4h713cg8w0jmrm5_tfzc0000gn/T/tmpv7hb703u/src.macosx-12-x86_64-3.10/norm1-f2pywrappers.f
INFO: /usr/local/bin/gfortran -Wall -g -Wall -g -undefined dynamic_lookup -bundle /var/folders/ws/5h_x4h713cg8w0jmrm5_tfzc0000gn/T/tmpv7hb703u/var/folders/ws/5h_x4h713cg8w0jmrm5_tfzc0000gn/T/tmpv7hb703u/src.macosx-12-x86_64-3.10/norm1module.o /var/folders/ws/5h_x4h713cg8w0jmrm5_tfzc0000gn/T/tmpv7hb703u/var/folders/ws/5h_x4h713cg8w0jmrm5_tfzc0000gn/T/tmpv7hb703u/src.macosx-12-x86_64-3.10/fortranobject.o /var/folders/ws/5h_x4h713cg8w0jmrm5_tfzc0000gn/T/tmpv7hb703u/book/tmp/vector_norm.o /var/folders/ws/5h_x4h713cg8w0jmrm5_tfzc0000gn/T/tmpv7hb703u/var/folders/ws/5h_x4h713cg8w0jmrm5_tfzc0000gn/T/tmpv7hb703u/src.macosx-12-x86_64-3.10/norm1-f2pywrappers.o -L/usr/local/Cellar/gcc/11.3.0_1/bin/../lib/gcc/11/gcc/x86_64-apple-darwin21/11 -L/usr/local/Cellar/gcc/11.3.0_1/bin/../lib/gcc/11/gcc/x86_64-apple-darwin21/11/../../.. -L/usr/local/Cellar/gcc/11.3.0_1/bin/../lib/g

In [28]:
import norm1
dir(norm1)

['__doc__',
 '__f2py_numpy_version__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__version__',
 '_norm1_error',
 'vector_norm']

In [31]:
import numpy as np
v = np.linspace(0,10)
norm1.vector_norm(v)

41.0325927734375

Looks like that works nocely, and the interface between Python and Fortran is much smoother!